In [48]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb 

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [50]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [51]:
df=pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
      df[feat + SUFFIX_CAT]=factorized_values


In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(x.split(' ')[0]) )
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(x.split('cm3')[0].replace(' ','')) )


In [0]:
def run_model(model,feats):
  x=df[feats].values
  y=df['price_value'].values

  scores = cross_val_score(model,x,y,cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [0]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

In [0]:
feats =['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']


In [65]:
run_model(xgb.XGBRegressor(**xgb_params ),feats)

[18:40:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-9569.227198767323 72.83561801421891


In [0]:
#m = xgb.XGBRegressor(**xgb_params )
#x=df[feats].values
#y=df['price_value'].values
#m.fit(x,y)

#imp = PermutationImportance(m, random_state=0).fit(x,y)
#eli5.show_weights(imp, feature_names=feats)

[16:41:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.5934 ± 0.0146,param_rok-produkcji
0.2553 ± 0.0033,param_moc
0.0517 ± 0.0007,param_marka-pojazdu__cat
0.0489 ± 0.0011,param_pojemność-skokowa
0.0288 ± 0.0005,seller_name__cat
0.0184 ± 0.0003,param_model-pojazdu__cat
0.0180 ± 0.0005,param_wersja__cat
0.0157 ± 0.0003,param_napęd__cat
0.0145 ± 0.0005,param_stan__cat
0.0144 ± 0.0004,param_typ__cat


In [0]:
def obj_func(params):
  print("Trenuj z parametrami:")
  print(params)

  mean_mae,score_std = run_model(xgb.XGBRegressor(**params),feats)
  print('MAE: {}'.format(np.abs(mean_mae)))
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space
xgb_reg_params = { 
    'learning_rate': hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth':  hp.choice('max_depth', np.arange(5,16,1, dtype= int)),
    'subsample':  hp.quniform('subsample', 0.5,1,0.05 ),
    'colsample_bytree':  hp.quniform('colsample_bytree', 0.5,1,0.05 ),
    'objective':  'reg:squarederror',
    'n_estimators': 50,
    'seed': 0}            

In [68]:
best = fmin(obj_func,xgb_reg_params,algo=tpe.suggest, max_evals=25)
best

Trenuj z parametrami:
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 50, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
MAE: 8314.960067508619
Trenuj z parametrami:
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 50, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
MAE: 8591.444052783767
Trenuj z parametrami:
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 50, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
MAE: 8158.815943850105
Trenuj z parametrami:
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 15, 'n_estimators': 50, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
MAE: 8720.621667180743
Trenuj z parametrami:
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 13, 'n_estimators': 50, 'objective': 'reg:squarederror', 's

{'colsample_bytree': 0.8, 'learning_rate': 4, 'max_depth': 7, 'subsample': 1.0}